<h2>
    Find the total number of messages exchanged between each person per day.
</h2>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr,col

spark=SparkSession.builder.appName('Message-Analysis').getOrCreate()

In [34]:
from pyspark.sql.functions import to_date,sum

data = [ ('2020-04-01', 'Avinash', 'Vibhor',10), 
        ('2020-04-01', 'Vibhor', 'Avinash',20), 
        ('2020-04-01', 'Avinash', 'Pawan',30), 
        ('2020-04-01', 'Pawan', 'Avinash',20), 
        ('2020-04-01', 'Vibhor', 'Pawan',5),
       ('2020-04-01', 'Pawan', 'Vibhor',8),
       ('2020-04-01', 'Vibhor', 'Deepak',50)] 
schema = """sms_date string,
 sender string,
 receiver string,
 sms_no int"""
df=spark.createDataFrame(data,schema)
df=df.withColumn("sms_date",to_date(df.sms_date))
df.printSchema()

root
 |-- sms_date: date (nullable = true)
 |-- sender: string (nullable = true)
 |-- receiver: string (nullable = true)
 |-- sms_no: integer (nullable = true)



In [38]:
person_df=df.withColumn("person1",expr("case when sender < receiver then sender else receiver end"))\
.withColumn("person2",expr("case when sender > receiver then sender else receiver end")).\
groupBy(col("sms_date"),col("person1"),col("person2")).\
agg(sum(col("sms_no")).alias("total_sms")).show()

+----------+-------+-------+---------+
|  sms_date|person1|person2|total_sms|
+----------+-------+-------+---------+
|2020-04-01|Avinash| Vibhor|       30|
|2020-04-01|Avinash|  Pawan|       50|
|2020-04-01|  Pawan| Vibhor|       13|
|2020-04-01| Deepak| Vibhor|       50|
+----------+-------+-------+---------+

